In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString
from ST import printSymTab

def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

Importing Jupyter notebook from P0.ipynb
Importing Jupyter notebook from SC.ipynb
Importing Jupyter notebook from ST.ipynb


### Scanner Tests for Exception

In [3]:
import SC
def scanString(src):
    SC.init(src); syms = []
    while SC.sym != SC.EOF:
        syms.append(('INDENT' if SC.sym == SC.INDENT else \
                     'DEDENT' if SC.sym == SC.DEDENT else \
                     'IDENT' if SC.sym == SC.IDENT else SC.sym, SC.newline))
        SC.getSym()
    return syms

#### Parsing Explicit Exceptions

In [7]:
scanString("""

program p

  if a then
    writeln()
  else
    writeln()
  if a then writeln() else writeln()
""")

[(48, True),
 ('IDENT', False),
 ('INDENT', False),
 (38, True),
 ('IDENT', False),
 (39, False),
 ('INDENT', False),
 ('IDENT', True),
 (23, False),
 (24, False),
 ('DEDENT', False),
 (40, True),
 ('INDENT', True),
 ('IDENT', True),
 (23, False),
 (24, False),
 ('DEDENT', False),
 (38, True),
 ('IDENT', False),
 (39, False),
 ('IDENT', False),
 (23, False),
 (24, False),
 (40, False),
 ('IDENT', False),
 (23, False),
 (24, False),
 ('DEDENT', False)]

In [8]:
scanString("""

program p

  if a then
    throw
  else
    throw
  if a then writeln() else writeln()
""")

[(48, True),
 ('IDENT', False),
 ('INDENT', False),
 (38, True),
 ('IDENT', False),
 (39, False),
 ('INDENT', False),
 (52, True),
 ('DEDENT', True),
 (40, True),
 ('INDENT', True),
 (52, True),
 ('DEDENT', True),
 (38, True),
 ('IDENT', False),
 (39, False),
 ('IDENT', False),
 (23, False),
 (24, False),
 (40, False),
 ('IDENT', False),
 (23, False),
 (24, False),
 ('DEDENT', False)]

In [ ]:
scanString("""

program p

  if a then
    throw exception('Just a test')
  else
    throw
  if a then throw exception('Just a test') else writeln()
""")

### Compiler Tests for Exception

In [5]:
# "Throw" can be compiled，but the WebAssembly Code is incorrect
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw else r := x
program equationsolver
    var a: integer
        a ← sqrt(2)
        write(a)
""")

Importing Jupyter notebook from CGwat.ipynb
(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(func $sqrt (param $x i32) (result i32)
(local $r i32)
(local $0 i32)
local.get $x
i32.const 0
i32.lt_s
if
throw 0
else
local.get $x
local.set $r
end
local.get $r)
(func $program
(local $a i32)
(local $0 i32)
i32.const 2
call $sqrt
local.set $a
local.get $a
call $write
)
(memory 1)
(start $program)
)


In [ ]:
!wat2wasm test.wat

#### Zero Division Exception

In [ ]:
compileString("""

program DividedByZero
    var a, b: integer
        a := (34 * 5 - 170)
        b := (99 div a)
        write(a)
""")

#### Access Before Initialization Exception

In [ ]:
compileString("""

program AccessBeforeInitialization
    a := 5
    var a: integer
    
""")

#### Wrong Type Exception

In [ ]:
compileString("""

program WrongType
    a := 5
    var a: boolean
    
""")

#### Variable Name Not Defined Exception

In [ ]:
compileString("""

program bIsNotDefined
    var a: boolean
    b := 5
    write(b)
""")

#### Index Out of Bounds Exception

In [ ]:
compileString("""

var x: [2 .. 4] → integer
program IndexOutOfBounds
  x[7] := 5
  
""")

#### Syntax Error Exception

In [ ]:
compileString("""

program missingProcedure
  var b: boolean
    b ← integer()
    
""")

In [ ]:
compileString("""

program incompatibleCall
  var b: boolean
    b ← read()
    
""")

In [ ]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw else ...
procedure quadraticsolution(a, b, c: integer) → (x, y: integer)
    var d: integer
        d ← sqrt(a × a - 4 × a × c)
        x, y := (- b + d) div (2 × a), (- b - d) div (2 × a)
program equationsolver
    var a, b, c: integer; done: boolean
        done := false
        while ¬done do
            try
                a ← read(); b ← read(); c ← read()
                x, y ← quadraticsolution(a, b, c)
                write(x); write(y)
           catch done := true
""")